In [2]:
!pip install xgboost
!pip install lightgbm
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.9 MB/s eta 0:00:00


In [7]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=3f5ba554be3cbe1159ae80e8ed9061074798b47c17434761cb83a287b41200b5
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


# **Codigo para compartir**

#########################################################################################

In [ ]:
# ===========================
# VECM con train-test split, pronósticos y métricas
# ===========================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM

# ===========================
# 1. Cargar y preparar datos
# ===========================
df = pd.read_excel("baseVECMfinal.xlsx")

tcr = "ITCER"
fundamentales = [
    "IPC", "energia", "agua", "gasliquido",
    "X", "M", "RIN",
    "activa", "pasivaahorro", "pasivafijo", "libor3", "FEDFUNDS",
    "EGRESOSCORRIENTES", "INGRESOSCORRIENTES", "EGRESOSCAPITAL", "INGRESOSCAPITAL",
    "Oro", "Petroleo1", "Zinc", "Plata", "Estano",
    "temperatura", "precipitation", "drought"
]

model_df = df[[tcr] + fundamentales].dropna()

# Variables que van en log
log_vars = ["ITCER","IPC","energia","agua","gasliquido","X","M","RIN",
            "Oro","Petroleo1","Zinc","Plata","Estano"]

for var in log_vars:
    model_df["ln_" + var] = np.log(model_df[var])

# Dataset final
Y = model_df[["ln_ITCER","ln_IPC","ln_X","ln_M","ln_RIN","ln_Oro","ln_Petroleo1",
              "ln_Zinc","ln_Plata","ln_Estano","activa","pasivaahorro","pasivafijo",
              "libor3","FEDFUNDS","EGRESOSCORRIENTES","INGRESOSCORRIENTES",
              "EGRESOSCAPITAL","INGRESOSCAPITAL","temperatura","precipitation","drought"]]

# ===========================
# 2. Train-test split
# ===========================
train_size = int(len(Y) * 0.8)
train, test = Y.iloc[:train_size], Y.iloc[train_size:]

# ===========================
# 3. Prueba de cointegración en train
# ===========================
johansen_test = coint_johansen(train, det_order=0, k_ar_diff=2)
print("Trace test:", johansen_test.lr1)
print("Critical values:", johansen_test.cvt)

# ===========================
# 4. Estimación VECM en train
# ===========================
vecm = VECM(train, k_ar_diff=2, coint_rank=1, deterministic="co")
vecm_res = vecm.fit()
print(vecm_res.summary())

# ===========================
# 5. Pronósticos
# ===========================
# Pronóstico en horizonte de test
n_test = len(test)
forecast_test = vecm_res.predict(steps=n_test)
forecast_test_df = pd.DataFrame(forecast_test,
                                index=test.index,
                                columns=Y.columns)

# Pronóstico extendido 78 pasos adelante
forecast_78 = vecm_res.predict(steps=78)
forecast_78_df = pd.DataFrame(forecast_78,
                              columns=Y.columns)

# ===========================
# 6. Guardar en Excel
# ===========================
with pd.ExcelWriter("pronosticos_VECM.xlsx") as writer:
    forecast_test_df.to_excel(writer, sheet_name="Forecast_Test")
    forecast_78_df.to_excel(writer, sheet_name="Forecast_78")

# ===========================
# 7. Métricas en variable clave (ln_ITCER)
# ===========================
aligned = pd.concat([test["ln_ITCER"], forecast_test_df["ln_ITCER"]], axis=1).dropna()
aligned.columns = ["y_true", "y_pred"]

y_true = aligned["y_true"].values
y_pred = aligned["y_pred"].values

# Métricas
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)  # corregido
mae = mean_absolute_error(y_true, y_pred)

with np.errstate(divide='ignore', invalid='ignore'):
    mape = np.mean(np.abs((y_true - y_pred) / y_true))

r2 = r2_score(y_true, y_pred)

print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.6f}")
print(f"MSE: {mse:.6f}")
print(f"MAE: {mae:.6f}")
print(f"MAPE: {mape*100:.4f}%")

# ===========================
# 8. Gráfico comparativo
# ===========================
plt.figure(figsize=(12,6))
plt.plot(train.index, train["ln_ITCER"], label="Train", color="blue")
plt.plot(aligned.index, aligned["y_true"], label="Test Real", color="black")
plt.plot(aligned.index, aligned["y_pred"], label="Pronóstico", linestyle="--", color="red")
plt.title("Pronóstico VECM vs Valores Reales (ln_ITCER)")
plt.xlabel("Tiempo")
plt.ylabel("ln_ITCER")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ===========================
# ECM (Regresión Lineal) con Train-Test, Pronósticos y Métricas
# ===========================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ===========================
# 1. Cargar y preparar datos
# ===========================
df = pd.read_excel("baseVECMfinal.xlsx")

tcr = "ITCER"
fundamentales = [
    "IPC", "energia", "agua", "gasliquido",
    "X", "M", "RIN",
    "activa", "pasivaahorro", "pasivafijo", "libor3", "FEDFUNDS",
    "EGRESOSCORRIENTES", "INGRESOSCORRIENTES", "EGRESOSCAPITAL", "INGRESOSCAPITAL",
    "Oro", "Petroleo1", "Zinc", "Plata", "Estano",
    "temperatura", "precipitation", "drought"
]

model_df = df[[tcr] + fundamentales].dropna()

# Variables en log
log_vars = ["ITCER","IPC","energia","agua","gasliquido","X","M","RIN",
            "Oro","Petroleo1","Zinc","Plata","Estano"]

for var in log_vars:
    model_df["ln_" + var] = np.log(model_df[var])

# Dataset: dependiente + regresores
Y = model_df["ln_ITCER"]
X = model_df.drop(columns=[tcr, "ITCER", "ln_ITCER"])  # quitamos duplicados

# ===========================
# 2. Train-test split
# ===========================
train_size = int(len(X) * 0.8)
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
Y_train, Y_test = Y.iloc[:train_size], Y.iloc[train_size:]

# ===========================
# 3. Estimación de regresión lineal
# ===========================
model = LinearRegression()
model.fit(X_train, Y_train)

# ===========================
# 4. Pronósticos
# ===========================
Y_pred = model.predict(X_test)

# ===========================
# 5. Métricas
# ===========================
mse = mean_squared_error(Y_test, Y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

with np.errstate(divide='ignore', invalid='ignore'):
    mape = np.mean(np.abs((Y_test.values - Y_pred) / Y_test.values))

print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.6f}")
print(f"MSE: {mse:.6f}")
print(f"MAE: {mae:.6f}")
print(f"MAPE: {mape*100:.4f}%")

# ===========================
# 6. Guardar en Excel
# ===========================
results_df = pd.DataFrame({
    "Real": Y_test.values,
    "Pronosticado": Y_pred
}, index=Y_test.index)

with pd.ExcelWriter("pronosticos_ECM.xlsx") as writer:
    results_df.to_excel(writer, sheet_name="Forecast_Test")
    pd.DataFrame({
        "R2":[r2], "RMSE":[rmse], "MSE":[mse], "MAE":[mae], "MAPE":[mape]
    }).to_excel(writer, sheet_name="Metrics", index=False)

# ===========================
# 7. Gráfico comparativo
# ===========================
plt.figure(figsize=(12,6))
plt.plot(Y_train.index, Y_train, label="Train", color="blue")
plt.plot(Y_test.index, Y_test, label="Test Real", color="black")
plt.plot(Y_test.index, Y_pred, label="Pronóstico", linestyle="--", color="red")
plt.title("Regresión Lineal (ECM) - Pronóstico vs Real (ln_ITCER)")
plt.xlabel("Tiempo")
plt.ylabel("ln_ITCER")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ============================
# Predicción con Modelos - Versión Mejorada
# Incluye: Validación temporal, SHAP, LIME, IC para el mejor modelo
# ============================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# =======================
# 1. Cargar y preparar datos
# =======================
file_path = "nueva_base_con_rezagosfinal.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

print("=== INFORMACIÓN DE LOS DATOS ===")
print(f"Dimensión original: {df.shape}")

# Eliminar filas con valores nulos en la variable objetivo
df = df.dropna(subset=["ITCRM"])
print(f"Dimensión después de limpiar NaN: {df.shape}")

# Crear índice temporal
if 'fecha' in df.columns or 'Fecha' in df.columns:
    date_col = 'fecha' if 'fecha' in df.columns else 'Fecha'
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col)
    df.index = df[date_col]
    print(f"Usando columna de fecha: {date_col}")
else:
    # Crear índice temporal automático
    dates = pd.date_range(start='1993-08-01', periods=len(df), freq='M')
    df.index = dates
    print("Índice temporal creado automáticamente")

print(f"Rango temporal: {df.index.min()} a {df.index.max()}")

# =======================
# 2. Análisis de estacionariedad
# =======================
from statsmodels.tsa.stattools import adfuller

def test_estacionariedad(serie, nombre="Serie"):
    """Test de Dickey-Fuller aumentado para estacionariedad"""
    result = adfuller(serie.dropna())
    p_value = result[1]
    es_estacionaria = p_value < 0.05

    print(f"\n{nombre}:")
    print(f"  p-value: {p_value:.6f}")
    print(f"  Estacionaria: {'SÍ' if es_estacionaria else 'NO'}")
    return es_estacionaria

# Test de estacionariedad
y_original = df["ITCRM"]
es_estacionaria = test_estacionariedad(y_original, "ITCRM Original")

if not es_estacionaria:
    print("\n⚠️  Aplicando diferenciación para lograr estacionariedad...")
    y_transformada = y_original.diff().dropna()
    es_estacionaria_transformada = test_estacionariedad(y_transformada, "ITCRM Diferenciada")

    if es_estacionaria_transformada:
        y_final = y_transformada
        metodo = "diferenciada"
        print("✅ Usando serie diferenciada (estacionaria)")
    else:
        y_final = y_original
        metodo = "original"
        print("⚠️  Usando serie original (no estacionaria)")
else:
    y_final = y_original
    metodo = "original"
    print("✅ Usando serie original (estacionaria)")

# =======================
# 3. Preparar variables predictoras
# =======================
X = df.loc[y_final.index].select_dtypes(include=[np.number]).drop(columns=["ITCRM"], errors='ignore')

# Limpiar variables con muchos NaN
umbral_nan = 0.3
X_clean = X.dropna(axis=1, thresh=int(len(X) * (1 - umbral_nan)))

# Imputar NaN restantes
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X_clean),
                        columns=X_clean.columns,
                        index=X_clean.index)

print(f"\n=== DATOS FINALES ===")
print(f"Observaciones: {len(X_imputed)}")
print(f"Variables predictoras: {len(X_imputed.columns)}")
print(f"Serie: {metodo}")

# =======================
# 4. Validación cruzada temporal para TODOS los modelos
# =======================
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Configurar modelos optimizados
modelos = {
    "RandomForest": RandomForestRegressor(
        n_estimators=200, max_depth=8, min_samples_split=15,
        min_samples_leaf=10, random_state=42, n_jobs=-1
    ),
    "XGBoost": xgb.XGBRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.1,
        random_state=42, n_jobs=-1
    ),
    "LightGBM": lgb.LGBMRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.1,
        random_state=42, n_jobs=-1, verbosity=-1
    ),
    "CatBoost": cb.CatBoostRegressor(
        iterations=200, depth=6, learning_rate=0.1,
        random_state=42, verbose=0
    )
}

# Validación cruzada temporal
n_splits = 5
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=int(len(X_imputed) * 0.2))

resultados_cv = {nombre: [] for nombre in modelos.keys()}

print(f"\n=== VALIDACIÓN CRUZADA TEMPORAL ({n_splits} folds) ===")

for fold, (train_idx, test_idx) in enumerate(tscv.split(X_imputed)):
    print(f"\n--- FOLD {fold + 1}/{n_splits} ---")

    X_train, X_test = X_imputed.iloc[train_idx], X_imputed.iloc[test_idx]
    y_train, y_test = y_final.iloc[train_idx], y_final.iloc[test_idx]

    for nombre, modelo in modelos.items():
        try:
            # Entrenar y predecir
            modelo.fit(X_train, y_train)
            y_pred = modelo.predict(X_test)

            # Métricas
            mae = mean_absolute_error(y_test, y_pred)
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            r2 = r2_score(y_test, y_pred)

            resultados_cv[nombre].append({
                "fold": fold + 1, "MAE": mae, "RMSE": rmse, "R2": r2
            })

            print(f"{nombre:12} - RMSE: {rmse:.4f}, R²: {r2:.4f}")

        except Exception as e:
            print(f"Error en {nombre}: {e}")

# =======================
# 5. Entrenamiento FINAL de TODOS los modelos
# =======================
print("\n" + "="*60)
print("ENTRENAMIENTO FINAL DE TODOS LOS MODELOS")
print("="*60)

modelos_entrenados = {}
predicciones_in_sample = {}
importancias_modelos = {}

for nombre, modelo in modelos.items():
    print(f"Entrenando {nombre}...")
    modelo.fit(X_imputed, y_final)
    modelos_entrenados[nombre] = modelo

    # Predicciones in-sample
    y_pred = modelo.predict(X_imputed)
    predicciones_in_sample[nombre] = y_pred

    # Importancia de variables para cada modelo
    try:
        if nombre == "CatBoost":
            importancias = modelo.get_feature_importance()
        elif nombre == "XGBoost":
            importancias = modelo.feature_importances_
        elif nombre == "LightGBM":
            importancias = modelo.feature_importances_
        elif nombre == "RandomForest":
            importancias = modelo.feature_importances_
        else:
            importancias = np.zeros(len(X_imputed.columns))

        importancias_modelos[nombre] = pd.DataFrame({
            'Variable': X_imputed.columns,
            'Importancia': importancias
        }).sort_values('Importancia', ascending=False)

    except Exception as e:
        print(f"Error calculando importancia para {nombre}: {e}")
        importancias_modelos[nombre] = pd.DataFrame()

# =======================
# 6. Análisis de resultados de TODOS los modelos
# =======================
print("\n" + "="*60)
print("RESULTADOS PROMEDIO - TODOS LOS MODELOS (Validación Cruzada)")
print("="*60)

resultados_promedio = {}
for modelo, folds in resultados_cv.items():
    if folds:
        df_folds = pd.DataFrame(folds)
        avg = df_folds.mean()
        std = df_folds.std()

        resultados_promedio[modelo] = {
            'MAE_avg': avg['MAE'], 'MAE_std': std['MAE'],
            'RMSE_avg': avg['RMSE'], 'RMSE_std': std['RMSE'],
            'R2_avg': avg['R2'], 'R2_std': std['R2']
        }

        print(f"\n{modelo}:")
        print(f"  RMSE: {avg['RMSE']:.4f} (±{std['RMSE']:.4f})")
        print(f"  R²:   {avg['R2']:.4f} (±{std['R2']:.4f})")

# Seleccionar mejor modelo
mejor_modelo_nombre = min(resultados_promedio.items(),
                         key=lambda x: x[1]['RMSE_avg'])[0]
mejor_modelo = modelos_entrenados[mejor_modelo_nombre]
print(f"\n🎯 MEJOR MODELO: {mejor_modelo_nombre}")

# =======================
# 7. CÁLCULO DE INTERVALOS DE CONFIANZA para el MEJOR MODELO
# =======================
print("\n" + "="*60)
print("CÁLCULO DE INTERVALOS DE CONFIANZA - MEJOR MODELO")
print("="*60)

def calcular_intervalos_confianza(modelo, X, y_real, metodo='residual', n_bootstraps=1000, alpha=0.05):
    """
    Calcula intervalos de confianza usando diferentes métodos
    """
    print(f"Método utilizado: {metodo}")

    # Predicción puntual
    y_pred = modelo.predict(X)
    residuales = y_real - y_pred

    if metodo == 'residual':
        # Método 1: Basado en residuales (aproximado)
        std_residual = np.std(residuales)
        z_score = 1.96  # Para 95% de confianza

        ic_inferior = y_pred - z_score * std_residual
        ic_superior = y_pred + z_score * std_residual

        print(f"Desviación estándar de residuales: {std_residual:.4f}")

    elif metodo == 'bootstrap':
        # Método 2: Bootstrap (más robusto)
        print(f"Ejecutando bootstrap con {n_bootstraps} iteraciones...")
        predicciones_bootstrap = []

        for i in range(n_bootstraps):
            # Remuestreo con reemplazo
            indices = np.random.choice(len(X), len(X), replace=True)
            X_boot = X.iloc[indices]
            y_boot = y_real.iloc[indices]

            # Re-entrenar modelo (copia para no afectar el original)
            modelo_boot = clone_modelo(modelo)
            modelo_boot.fit(X_boot, y_boot)

            # Predecir en datos originales
            y_pred_boot = modelo_boot.predict(X)
            predicciones_bootstrap.append(y_pred_boot)

        predicciones_bootstrap = np.array(predicciones_bootstrap)
        ic_inferior = np.percentile(predicciones_bootstrap, (alpha/2)*100, axis=0)
        ic_superior = np.percentile(predicciones_bootstrap, (1-alpha/2)*100, axis=0)

    else:
        raise ValueError("Método no reconocido")

    # Calcular cobertura empírica (si tenemos datos reales)
    cobertura = np.mean((y_real >= ic_inferior) & (y_real <= ic_superior))
    ancho_promedio = np.mean(ic_superior - ic_inferior)

    print(f"Cobertura empírica del IC: {cobertura:.3f}")
    print(f"Ancho promedio del IC: {ancho_promedio:.4f}")

    return y_pred, ic_inferior, ic_superior, residuales

def clone_modelo(modelo_original):
    """Crea una copia del modelo para bootstrap"""
    if hasattr(modelo_original, 'copy'):
        return modelo_original.copy()
    else:
        # Para modelos que no tienen método copy, recrear con mismos parámetros
        if isinstance(modelo_original, RandomForestRegressor):
            return RandomForestRegressor(**modelo_original.get_params())
        elif isinstance(modelo_original, xgb.XGBRegressor):
            return xgb.XGBRegressor(**modelo_original.get_params())
        elif isinstance(modelo_original, lgb.LGBMRegressor):
            return lgb.LGBMRegressor(**modelo_original.get_params())
        elif isinstance(modelo_original, cb.CatBoostRegressor):
            return cb.CatBoostRegressor(**modelo_original.get_params())
        else:
            return modelo_original

# Calcular intervalos de confianza para el mejor modelo
y_pred_mejor, ic_inf, ic_sup, residuales_mejor = calcular_intervalos_confianza(
    mejor_modelo, X_imputed, y_final, metodo='residual'
)

# =======================
# 8. Pronóstico fuera de muestra con IC para el MEJOR MODELO
# =======================
print("\n" + "="*60)
print("PRONÓSTICO FUERA DE MUESTRA CON IC - MEJOR MODELO")
print("="*60)

predicciones_out_sample = {}
ic_out_sample = {}

# Crear conjunto para pronóstico fuera de muestra (últimos 74 meses)
if len(X_imputed) > 74:
    X_train_final = X_imputed.iloc[:-74]
    y_train_final = y_final.iloc[:-74]
    X_test_final = X_imputed.iloc[-74:]
    y_test_final = y_final.iloc[-74:]

    # Re-entrenar mejor modelo con datos hasta el punto de corte
    mejor_modelo.fit(X_train_final, y_train_final)

    # Pronóstico puntual
    y_pred_out = mejor_modelo.predict(X_test_final)
    predicciones_out_sample[mejor_modelo_nombre] = y_pred_out

    # Calcular IC para pronóstico fuera de muestra
    residuales_train = y_train_final - mejor_modelo.predict(X_train_final)
    std_residual_out = np.std(residuales_train)

    ic_inf_out = y_pred_out - 1.96 * std_residual_out
    ic_sup_out = y_pred_out + 1.96 * std_residual_out

    ic_out_sample[mejor_modelo_nombre] = {
        'inferior': ic_inf_out,
        'superior': ic_sup_out
    }

    print(f"Pronóstico fuera de muestra calculado para {len(y_pred_out)} meses")
    print(f"Desviación estándar residuales (train): {std_residual_out:.4f}")

else:
    print("Datos insuficientes para pronóstico fuera de muestra")

# =======================
# 9. ANÁLISIS SHAP solo del MEJOR modelo
# =======================
print("\n" + "="*60)
print("ANÁLISIS SHAP - SOLO MEJOR MODELO")
print("="*60)

shap_importance = pd.DataFrame()
lime_df = pd.DataFrame()

try:
    import shap

    # Calcular valores SHAP
    explainer = shap.TreeExplainer(mejor_modelo)
    shap_values = explainer.shap_values(X_imputed)

    # Importancia global
    shap_importance = pd.DataFrame({
        'Variable': X_imputed.columns,
        'SHAP_Importance': np.abs(shap_values).mean(axis=0)
    }).sort_values('SHAP_Importance', ascending=False)

    print("\nTop 15 Variables Más Importantes (SHAP) - Mejor Modelo:")
    for i, row in shap_importance.head(15).iterrows():
        print(f"  {i+1:2d}. {row['Variable']:30} {row['SHAP_Importance']:.6f}")

    # Gráfico SHAP
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_imputed, plot_type="bar", show=False)
    plt.title(f'SHAP Feature Importance - {mejor_modelo_nombre}', fontsize=14)
    plt.tight_layout()
    plt.savefig('shap_importance.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Gráfico SHAP guardado como 'shap_importance.png'")

except Exception as e:
    print(f"❌ Error en SHAP: {e}")

# =======================
# 10. ANÁLISIS LIME solo del MEJOR modelo
# =======================
print("\n" + "="*60)
print("ANÁLISIS LIME - SOLO MEJOR MODELO")
print("="*60)

try:
    from lime import lime_tabular

    # Preparar explainer LIME
    lime_explainer = lime_tabular.LimeTabularExplainer(
        training_data=np.array(X_imputed),
        feature_names=X_imputed.columns.tolist(),
        mode='regression',
        verbose=False,
        random_state=42
    )

    # Analizar las primeras 3 observaciones
    lime_results = []
    n_explicaciones = min(3, len(X_imputed))

    for i in range(n_explicaciones):
        exp = lime_explainer.explain_instance(
            data_row=X_imputed.iloc[i].values,
            predict_fn=mejor_modelo.predict,
            num_features=10
        )

        # Convertir explicación a DataFrame
        explanation_df = pd.DataFrame(exp.as_list(), columns=['Variable', 'Impacto'])
        explanation_df['Observacion'] = i
        explanation_df['Fecha'] = X_imputed.index[i]
        explanation_df['Modelo'] = mejor_modelo_nombre
        lime_results.append(explanation_df)

        print(f"\nObservación {i+1} ({X_imputed.index[i].strftime('%Y-%m')}):")
        for _, row in explanation_df.head(5).iterrows():
            print(f"  {row['Variable']:30} {row['Impacto']:+.6f}")

    lime_df = pd.concat(lime_results, ignore_index=True)
    print("✅ Análisis LIME completado")

except Exception as e:
    print(f"❌ Error en LIME: {e}")

# =======================
# 11. GRÁFICOS COMPLETOS con INTERVALOS DE CONFIANZA
# =======================
print("\n" + "="*60)
print("GENERANDO GRÁFICOS CON INTERVALOS DE CONFIANZA")
print("="*60)

# Gráfico 1: Serie real vs predicciones con IC del mejor modelo
plt.figure(figsize=(16, 12))

# Subplot 1: Serie real vs predicciones con IC (in-sample)
plt.subplot(2, 2, 1)
plt.plot(y_final.index, y_final.values, label='Real', linewidth=2, alpha=0.9, color='black')
plt.plot(y_final.index, y_pred_mejor, label=f'Predicho ({mejor_modelo_nombre})',
         linewidth=1.5, alpha=0.8, color='red')

# Rellenar área del intervalo de confianza
plt.fill_between(y_final.index, ic_inf, ic_sup, alpha=0.3, color='red',
                label='IC 95%')

plt.title(f'Serie Real vs Predicciones con IC\n({mejor_modelo_nombre})', fontsize=12)
plt.xlabel('Fecha')
plt.ylabel('ITCRM')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Zoom a los últimos 24 meses con IC
plt.subplot(2, 2, 2)
ultimos_meses = 24
if len(y_final) > ultimos_meses:
    idx_ultimos = slice(-ultimos_meses, None)
    plt.plot(y_final.index[idx_ultimos], y_final.values[idx_ultimos],
             label='Real', linewidth=2, alpha=0.9, color='black', marker='o')
    plt.plot(y_final.index[idx_ultimos], y_pred_mejor[idx_ultimos],
             label=f'Predicho', linewidth=1.5, alpha=0.8, color='red', marker='s')
    plt.fill_between(y_final.index[idx_ultimos], ic_inf[idx_ultimos],
                    ic_sup[idx_ultimos], alpha=0.3, color='red', label='IC 95%')

    plt.title(f'Zoom últimos {ultimos_meses} meses con IC', fontsize=12)
    plt.xlabel('Fecha')
    plt.ylabel('ITCRM')
    plt.legend()
    plt.grid(True, alpha=0.3)

# Subplot 3: Pronóstico fuera de muestra con IC
if predicciones_out_sample:
    plt.subplot(2, 2, 3)
    y_pred_out = predicciones_out_sample[mejor_modelo_nombre]
    ic_inf_out = ic_out_sample[mejor_modelo_nombre]['inferior']
    ic_sup_out = ic_out_sample[mejor_modelo_nombre]['superior']

    plt.plot(y_test_final.index, y_test_final.values, label='Real',
             linewidth=2, alpha=0.9, color='black', marker='o')
    plt.plot(y_test_final.index, y_pred_out, label='Predicho',
             linewidth=1.5, alpha=0.8, color='blue', marker='s')
    plt.fill_between(y_test_final.index, ic_inf_out, ic_sup_out,
                    alpha=0.3, color='blue', label='IC 95%')

    plt.title('Pronóstico Fuera de Muestra con IC', fontsize=12)
    plt.xlabel('Fecha')
    plt.ylabel('ITCRM')
    plt.legend()
    plt.grid(True, alpha=0.3)

# Subplot 4: Distribución de residuales y cobertura IC
plt.subplot(2, 2, 4)
# Histograma de residuales
plt.hist(residuales_mejor, bins=30, alpha=0.7, color='green', edgecolor='black',
         density=True, label='Residuales')

# Líneas para ±1.96σ (límites del IC)
std_residual = np.std(residuales_mejor)
plt.axvline(-1.96 * std_residual, color='red', linestyle='--',
           label='±1.96σ (límites IC)')
plt.axvline(1.96 * std_residual, color='red', linestyle='--')

plt.title('Distribución Residuales y Límites IC', fontsize=12)
plt.xlabel('Residual')
plt.ylabel('Densidad')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('resultados_con_ic.png', dpi=300, bbox_inches='tight')
plt.close()
print("✅ Gráfico con intervalos de confianza guardado como 'resultados_con_ic.png'")

# =======================
# 12. GUARDAR RESULTADOS COMPLETOS EN EXCEL
# =======================
print("\n" + "="*60)
print("GUARDANDO RESULTADOS COMPLETOS EN EXCEL")
print("="*60)

with pd.ExcelWriter('resultados_completos_con_ic.xlsx') as writer:
    # 1. Metadatos del análisis
    metadata = pd.DataFrame({
        'Parámetro': ['Fecha análisis', 'Total observaciones', 'Variables predictoras',
                     'Mejor modelo', 'Transformación', 'Folds CV', 'Método IC', 'Cobertura IC'],
        'Valor': [datetime.now().strftime('%Y-%m-%d %H:%M'), len(X_imputed),
                 len(X_imputed.columns), mejor_modelo_nombre, metodo, n_splits,
                 'Residual', f"{np.mean((y_final >= ic_inf) & (y_final <= ic_sup)):.3f}"]
    })
    metadata.to_excel(writer, sheet_name='Metadatos', index=False)

    # 2. Resultados de validación cruzada (todos los modelos)
    resultados_cv_df = pd.DataFrame([
        {**{'modelo': model}, **metrics}
        for model, metrics in resultados_promedio.items()
    ])
    resultados_cv_df.to_excel(writer, sheet_name='Resultados_CV', index=False)

    # 3. Predicciones con IC del mejor modelo (in-sample)
    predicciones_ic_df = pd.DataFrame({
        'Fecha': y_final.index,
        'Real': y_final.values,
        'Predicho': y_pred_mejor,
        'IC_Inferior': ic_inf,
        'IC_Superior': ic_sup,
        'Residual': residuales_mejor,
        'Dentro_IC': (y_final >= ic_inf) & (y_final <= ic_sup)
    })
    predicciones_ic_df.to_excel(writer, sheet_name='Predicciones_IC_InSample', index=False)

    # 4. Pronóstico fuera de muestra con IC
    if predicciones_out_sample:
        predicciones_os_df = pd.DataFrame({
            'Fecha': y_test_final.index,
            'Real': y_test_final.values,
            'Predicho': predicciones_out_sample[mejor_modelo_nombre],
            'IC_Inferior': ic_out_sample[mejor_modelo_nombre]['inferior'],
            'IC_Superior': ic_out_sample[mejor_modelo_nombre]['superior'],
            'Residual': y_test_final.values - predicciones_out_sample[mejor_modelo_nombre]
        })
        predicciones_os_df.to_excel(writer, sheet_name='Predicciones_FueraMuestra', index=False)

    # 5. Importancia de variables (todos los modelos)
    for nombre, df_importancia in importancias_modelos.items():
        if not df_importancia.empty:
            df_importancia.to_excel(writer, sheet_name=f'Importancia_{nombre}', index=False)

    # 6. Resultados SHAP
    if not shap_importance.empty:
        shap_importance.to_excel(writer, sheet_name='SHAP_Importance', index=False)

    # 7. Resultados LIME
    if not lime_df.empty:
        lime_df.to_excel(writer, sheet_name='LIME_Explicaciones', index=False)

    # 8. Datos originales procesados
    datos_procesados = pd.DataFrame({
        'Fecha': X_imputed.index,
        'ITCRM': y_final.values
    })
    datos_procesados = pd.concat([datos_procesados, X_imputed], axis=1)
    datos_procesados.to_excel(writer, sheet_name='Datos_Procesados', index=False)

print("✅ Archivo Excel guardado como 'resultados_completos_con_ic.xlsx'")

# =======================
# 13. RESUMEN FINAL
# =======================
print("\n" + "="*60)
print("RESUMEN FINAL DEL ANÁLISIS")
print("="*60)

print(f"📊 Mejor modelo seleccionado: {mejor_modelo_nombre}")
print(f"📈 Métricas promedio (CV):")
print(f"   - RMSE: {resultados_promedio[mejor_modelo_nombre]['RMSE_avg']:.4f}")
print(f"   - R²: {resultados_promedio[mejor_modelo_nombre]['R2_avg']:.4f}")
print(f"📐 Intervalos de confianza: 95% (método residual)")
print(f"   - Cobertura empírica: {np.mean((y_final >= ic_inf) & (y_final <= ic_sup)):.3f}")
print(f"   - Ancho promedio: {np.mean(ic_sup - ic_inf):.4f}")
print(f"💾 Resultados guardados:")
print(f"   - resultados_completos_con_ic.xlsx (datos completos)")
print(f"   - resultados_con_ic.png (gráficos)")
print(f"   - shap_importance.png (importancia SHAP)")

print("\n🎯 ANÁLISIS COMPLETADO EXITOSAMENTE")

=== INFORMACIÓN DE LOS DATOS ===
Dimensión original: (370, 446)
Dimensión después de limpiar NaN: (370, 446)
Índice temporal creado automáticamente
Rango temporal: 1993-08-31 00:00:00 a 2024-05-31 00:00:00

ITCRM Original:
  p-value: 0.057772
  Estacionaria: NO

⚠️  Aplicando diferenciación para lograr estacionariedad...

ITCRM Diferenciada:
  p-value: 0.000000
  Estacionaria: SÍ
✅ Usando serie diferenciada (estacionaria)

=== DATOS FINALES ===
Observaciones: 369
Variables predictoras: 444
Serie: diferenciada

=== VALIDACIÓN CRUZADA TEMPORAL (5 folds) ===

--- FOLD 1/5 ---
RandomForest - RMSE: 0.0238, R²: -0.9479
XGBoost      - RMSE: 0.0216, R²: -0.6063
LightGBM     - RMSE: 0.0236, R²: -0.9130
CatBoost     - RMSE: 0.0234, R²: -0.8932

--- FOLD 2/5 ---
RandomForest - RMSE: 0.0251, R²: -0.0075
XGBoost      - RMSE: 0.0261, R²: -0.0868
LightGBM     - RMSE: 0.0261, R²: -0.0836
CatBoost     - RMSE: 0.0247, R²: 0.0245

--- FOLD 3/5 ---
RandomForest - RMSE: 0.0262, R²: 0.0360
XGBoost      - RM